In [1]:
import numpy as np
import anndata as ad
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [4]:
X=np.loadtxt('./inDrop1.txt')

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
"""
use the manhatten distance to make the kmeans algorithm

"""
def kmeans_manhattan(X, n_clusters, max_iter=30000, tol=1e-4, random_state=None):
    if random_state is not None:
        #set random seed
        np.random.seed(random_state)
    
    # randomly choose the cluster center 
    initial_indices = np.random.choice(X.shape[0], n_clusters, replace=False)
    cluster_centers = X[initial_indices]
    #iteration
    for i in range(max_iter):
        # calculate the manhatten distance to the closest cluster center and label them 
        labels = pairwise_distances(X, cluster_centers, metric='manhattan').argmin(axis=1)
        
        # Use these data to get the new cluster center
        new_centers = np.array([X[labels == j].mean(axis=0) for j in range(n_clusters)])
        
        # see if the centers aren't change much in 2 iterations
        if np.all(np.abs(new_centers - cluster_centers) < tol):
            break
        
        cluster_centers = new_centers

    return cluster_centers, labels


n_clusters = 14
cluster_centers, labels = kmeans_manhattan(X, n_clusters)
print(len(labels))
lb=[]
for i in range(len(labels)):
    lb.append(str(int(labels[i])))
lb=np.array(lb)
np.savetxt("./labelsKmeans.txt",lb,fmt="%s")



1937


In [5]:
from sklearn.cluster import KMeans
K=14
kmeans = KMeans(n_clusters=K, random_state=None)  
kmeans.fit(X)  
labels = kmeans.predict(X)  
print(len(labels))
lb=[]
for i in range(len(labels)):
    lb.append(str(int(labels[i])))
lb=np.array(lb)
np.savetxt("./labelsKmeans.txt",lb,fmt="%s")

C:\Users\rossoneri\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


1937


In [5]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

def knn_smoothing(data, n_neighbors=5):
    """
    Apply KNN smoothing to the data.

    Parameters:
    - data: 2D array-like, shape (n_samples, n_features)
    - n_neighbors: int, number of neighbors to use for smoothing

    Returns:
    - smoothed_data: 2D array, smoothed data
    """
    # Fit the NearestNeighbors model
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(data)
    distances, indices = nbrs.kneighbors(data)
    
    # Perform KNN smoothing
    smoothed_data = np.zeros_like(data)
    for i in range(data.shape[0]):
        smoothed_data[i, :] = np.mean(data[indices[i, :], :], axis=0)
    
    return smoothed_data

def classify_cells_with_knn_smoothing(cell_gene_matrix, n_clusters=3, n_neighbors=5):
    """
    Classifies cells using KNN smoothing and K-means clustering.

    Parameters:
    - cell_gene_matrix: list of lists, representing the cell*gene matrix.
    - n_clusters: int, number of clusters to use for K-means clustering.
    - n_neighbors: int, number of neighbors to use for KNN smoothing.

    Returns:
    - labels: list, predicted labels for each cell.
    """
    # Convert the cell*gene matrix to a DataFrame
    df = pd.DataFrame(cell_gene_matrix)
    
    # Apply KNN smoothing
    smoothed_data = knn_smoothing(df.values, n_neighbors=n_neighbors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(smoothed_data)
    
    # Get the cluster labels
    labels = kmeans.labels_
    
    return labels.tolist()



lb = classify_cells_with_knn_smoothing(X,n_clusters=14, n_neighbors=10)
np.savetxt("./labelsKmeans.txt",lb,fmt="%s")

C:\Users\rossoneri\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
